In [41]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import time

In [3]:
df = pd.read_excel('C:/Develops/newspapers/중앙일보/NewsResult_20200901-20210430 (8).xlsx', names=['identical', 'date', 'press', 'name', 'title', 'c1', 'c2', 'c3', 'a1', 'a2', 'a3', 'person', 'place', 'institute', 'keyword', 'topkeyword', 'body', 'url', 'tf'])

# 전처리
df = df[df.tf != '예외']
df = df[df.tf != '중복']
df = df[df.tf != '중복, 예외']
# df = df[~df.title.str.contains('경향포토')]
# df = df[~df.title.str.contains('인터랙티브')]
# df = df[~df.place.str.contains('korea', na=False)]
# df = df[~df.place.str.contains('la', na=False)]
# df = df[~df.place.str.contains('LA', na=False)]
df = df.reset_index()
df = df.drop(columns=['index'], axis=1)

In [5]:
# ~4/1 7952, ~3/1 16355
df = df.iloc[0:7952] # 4/30 ~ 4/1 1달간 총 7952개(8시간 50분) *50개(3분20초)

In [6]:
len(df)

7952

In [7]:
df

,identical,date,press,name,title,c1,c2,c3,a1,a2,a3,person,place,institute,keyword,topkeyword,body,url,tf
0,1.100901e+06,20210430,중앙일보,한영혜(han.younghye@joongang.co.kr),"머리하다 날벼락 80대 운전자 미용실로 車 돌진, 1명 사망",사회>사건_사고,국제>중남미,국제>유럽_EU,사고>교통사고>노상사고,NaN,NaN,NaN,"서울,서초동",경찰,"머리,날벼락,80대,운전자,미용실,돌진,사망,서울,서초동,남성,운전,승용차,미용실,...","미용실,1명,80대,a씨,승용차,목격자,날벼락,관계자,특례법,교통사고,1차,서울,골...",서울 서초동에서 80대 남성이 운전하던 승용차가 미용실 안으로 돌진해 손님 1명이 ...,https://news.joins.com/article/olink/23642701,NaN
1,1.100901e+06,20210430,중앙일보,김효경(kaypubb@joongang.co.kr),"하주석 5안타 6타점, 한화 롯데 꺾고 4연패 탈출",스포츠>야구,스포츠>농구_배구,스포츠>축구,NaN,NaN,NaN,"박세웅,윤호솔","대구,경기,사직구장,부산","한화 이글스,삼성 라이온즈,한화,롯데,LG 트윈스,라이온,롯데 자이언츠,LG,삼성","6타점,하주석,한화,롯데,탈출,무실점,윤호솔,이닝,9년,프로,이글스,프로,야구,한화...","하주석,한화,롯데,삼성,무실점,적시타,윤호솔,5안타,10패,4연패,2회,노시환,6타...",프로야구 한화 이글스가 하주석의 활약에 힘입어 롯데 자이언츠에 재역전승을 거뒀다. ...,https://news.joins.com/article/olink/23642699,NaN
2,1.100901e+06,20210430,중앙일보,한영혜(han.younghye@joongang.co.kr),"아파트 내서 '2m' 음주운전 1500만원 벌금폭탄 맞았다, 왜",사회>장애인,NaN,NaN,NaN,NaN,NaN,NaN,울산,"울산지법,형사5단독,재판부","2m,아파트,내서,음주운전,1500만,벌금폭탄,단지,아파트,가량,차량,운전,혐의,기...","음주운전,울산지법,a씨,교통사고,회사원,1500만,내서,김정철,혈중알콜농도,알코올농...",술을 마신 채 아파트 단지 내에서 2m가량 차량을 운전한 혐의로 기소된 50대 회사...,https://news.joins.com/article/olink/23642698,NaN
3,1.100901e+06,20210430,중앙일보,김준희(kim.junhee@joongang.co.kr),회식 모임 금지했는데 나주경찰서장과 간부 3명 골프 쳤다,지역>강원,지역>대전,지역>전남,NaN,NaN,NaN,NaN,"경찰서장,전남,나주","경찰서,전남경찰청,정부,영암,감찰계,나주경찰서장","회식,모임,나주경찰서장,골프,간부,전남경찰청,확인,전남경찰청,감찰,착수,전남,지역,...","전남경찰청,전남,나주,코로나바이러스,경찰서,나주경찰서장,3명,간부들,공무원들,사실관...",전남 지역 한 경찰서장과 간부들이 평일에 골프를 친 것으로 드러나 복무 지침 위반 ...,https://news.joins.com/article/olink/23642697,NaN
4,1.100901e+06,20210430,중앙일보,정은혜(jeong.eunhye1@joongang.co.kr),"화이자, EMA에 ""백신 접종 대상 12 15세까지 확대"" 요청",미분류,NaN,NaN,NaN,NaN,NaN,사힌,"위구르,독일,미국","EU,유럽의약품청,바이오엔테크사,EMA,AFP통신,유럽연합,제약사,화이자","EMA,대상,백신,접종,15세,확대,요청,화이자,미국,제약사,독일,바이오엔테크사,유...","미국,바이오엔테크사,15세,제약사,ema,유럽,eu,유럽의약품청,의약품청,연령대,유...",미국 제약사 화이자와 독일 바이오엔테크사가 유럽의약품청(EMA)에 자사 개발 백신(...,https://news.joins.com/article/olink/23642696,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7947,1.100901e+06,20210401,중앙일보,정진우(dino87@joongang.co.kr),대만 코앞서 열리는 한 중외교회담,정치>외교,정치>청와대,국제>중국,NaN,NaN,NaN,"서훈,기타무라 시게루,설리번,왕이,정의용","미국,일본,중국,헤네시닐랜드,팔라우,한국,태평양,한반도,중,장교,인도,아나폴리스,미...","국가안보국,중국,해군,한국,국가안보실,한·중외교회,정부,해군사관학교,청와대,한·미 ...","대만,코앞,중외교회담,중국,미국,장소,대결,상징,미국선,안보실장,협의,해군,쿼드,핵...","중국,대만,미국,샤먼,백악관,청와대,쿼드,일본,정의용,외교부,한국,아나폴리스,해병대...",바이든 정부 출범 이후 미 중 갈등이 고조되는 상황에서 정부가 외교 고차방정식을 풀...,https://news.joins.com/article/olink/23619867,NaN
7948,1.100901e+06,20210401,중앙일보,김홍범(kim.hongbum@joongang.co.kr),화이자 “12~15세 예방효과 100%” 청소년 접종 길 열린다,사회>의료_건강,IT_과학>IT_과학일반,NaN,NaN,NaN,NaN,앨버트 불라,"미국소아과학회,최고경영자,미국","바이오엔테크,FDA,화이자,식품의약국,AP 통신","100%,예방,효과,100%,청소년,접종,결과,미국,임상,결과,수주,허가,신청,가을...","미국,코로나19,월스트리트저널,불라,앨버트,경영자,의약국,6개월,11세,최고경영자,...",화이자 바이오엔테크의 코로나19 백신이 12 15세 청소년에게 100%의 예방 효과...,https://news.joins.com/article/olink/23619866,NaN
7949,1.100901e+06,20210401,중앙일보,강찬호(stoncold@joongang.co.kr),"“김정숙 여사가 끌어안길래, 밀어내며 쓴소리했다”",정치>외교,정치>청와대,NaN,NaN,NaN,NaN,"김,윤청자,김정숙,윤 여사,황기철,문재인,문,민평기","서해,김,천안,하길,북한,평택,평창","정찰총국,사령부,정부,중앙일보,북한,해군","김정숙,여사,쓴소리,아들,천안함,폭침,윤청자씨,북한,도발,숨기,묻자,미안,사령부,평...","추모식,천안함,옆자리,쓴소리,숨기,윤청자,평창,서해수호,평택,민평기,문재인,보훈처장...",지난 26일 평택 해군 2함대 사령부에서 열린 ‘제6회 서해수호의 날’ 기념식에서 ...,https://news.joins.com/article/olink/23619864,NaN
7950,1.100901e+06,20210401,중앙일보,신성식.김민욱(ssshin@joongang.co.kr),"2030은 설암, 60대는 편도암 늘었다",사회>의료_건강,지역>경기,지역>충남,NaN,NaN,NaN,"석준걸,이비인후과,정유석,정규원","설암,구강","보건복지부,중앙암등록본부,이비인후과,국립암센터,세계보건기구,WHO","설암,60대,편도암,국립암센터,중앙암,등록,통계,요인,유전,20,,설암,혀암,60대...","편도암,설암,연평균,국립암센터,60대,정유석,국가암검진사업,자궁경부암,헬리코박터균,...","20, 30대는 설암(혀암), 60대 이상은 편도암의 증가세가 두드러진 것으로 나타...",https://news.joins.com/article/olink/23619863,NaN


In [23]:
text_0_7953_넣음 = pd.read_excel('C:/Develops/newspapers/중앙일보/text_0_7953_넣음.xls', header=None)
len(text_0_7953_넣음)

7952

In [11]:
text_0_7953_넣음 = text_0_7953_넣음.values.tolist()

In [ ]:
judge['judge']

In [28]:
judge = pd.read_excel('C:/Develops/newspapers/중앙일보/judge.xls')
judge

,Unnamed: 0,judge,judge_ko
0,0,-1,-부정
1,1,-1,중립
2,2,0,중립
3,3,0,중립
4,4,0,중립
...,...,...,...
7947,7947,-1,-부정
7948,7948,0,중립
7949,7949,-1,-부정
7950,7950,-1,-부정


In [29]:
judge_value = judge['judge'].values.tolist()

In [30]:
judge_ko = judge['judge_ko'].values.tolist()

In [ ]:
# df['total_text'] = co_text

In [24]:
df['total_body'] = text_0_7953_넣음

In [31]:
df['judge'] = judge_value

In [32]:
df['judge_ko'] = judge_ko

In [33]:
df

,identical,date,press,name,title,c1,c2,c3,a1,a2,...,place,institute,keyword,topkeyword,body,url,tf,total_body,judge,judge_ko
0,1.100901e+06,20210430,중앙일보,한영혜(han.younghye@joongang.co.kr),"머리하다 날벼락 80대 운전자 미용실로 車 돌진, 1명 사망",사회>사건_사고,국제>중남미,국제>유럽_EU,사고>교통사고>노상사고,NaN,...,"서울,서초동",경찰,"머리,날벼락,80대,운전자,미용실,돌진,사망,서울,서초동,남성,운전,승용차,미용실,...","미용실,1명,80대,a씨,승용차,목격자,날벼락,관계자,특례법,교통사고,1차,서울,골...",서울 서초동에서 80대 남성이 운전하던 승용차가 미용실 안으로 돌진해 손님 1명이 ...,https://news.joins.com/article/olink/23642701,NaN,사진 크게보기 30일 오전 11시 25분께 80대 남성이 몰던 승용차가 미용실로 돌...,-1,-부정
1,1.100901e+06,20210430,중앙일보,김효경(kaypubb@joongang.co.kr),"하주석 5안타 6타점, 한화 롯데 꺾고 4연패 탈출",스포츠>야구,스포츠>농구_배구,스포츠>축구,NaN,NaN,...,"대구,경기,사직구장,부산","한화 이글스,삼성 라이온즈,한화,롯데,LG 트윈스,라이온,롯데 자이언츠,LG,삼성","6타점,하주석,한화,롯데,탈출,무실점,윤호솔,이닝,9년,프로,이글스,프로,야구,한화...","하주석,한화,롯데,삼성,무실점,적시타,윤호솔,5안타,10패,4연패,2회,노시환,6타...",프로야구 한화 이글스가 하주석의 활약에 힘입어 롯데 자이언츠에 재역전승을 거뒀다. ...,https://news.joins.com/article/olink/23642699,NaN,30일 롯데전에서 결승타를 때려내는 한화 하주석. [연합뉴스] 프로야...,-1,중립
2,1.100901e+06,20210430,중앙일보,한영혜(han.younghye@joongang.co.kr),"아파트 내서 '2m' 음주운전 1500만원 벌금폭탄 맞았다, 왜",사회>장애인,NaN,NaN,NaN,NaN,...,울산,"울산지법,형사5단독,재판부","2m,아파트,내서,음주운전,1500만,벌금폭탄,단지,아파트,가량,차량,운전,혐의,기...","음주운전,울산지법,a씨,교통사고,회사원,1500만,내서,김정철,혈중알콜농도,알코올농...",술을 마신 채 아파트 단지 내에서 2m가량 차량을 운전한 혐의로 기소된 50대 회사...,https://news.joins.com/article/olink/23642698,NaN,사진 크게보기 음주운전 단속 현장. 연합뉴스 술을 마신 채 아파트 단...,0,중립
3,1.100901e+06,20210430,중앙일보,김준희(kim.junhee@joongang.co.kr),회식 모임 금지했는데 나주경찰서장과 간부 3명 골프 쳤다,지역>강원,지역>대전,지역>전남,NaN,NaN,...,"경찰서장,전남,나주","경찰서,전남경찰청,정부,영암,감찰계,나주경찰서장","회식,모임,나주경찰서장,골프,간부,전남경찰청,확인,전남경찰청,감찰,착수,전남,지역,...","전남경찰청,전남,나주,코로나바이러스,경찰서,나주경찰서장,3명,간부들,공무원들,사실관...",전남 지역 한 경찰서장과 간부들이 평일에 골프를 친 것으로 드러나 복무 지침 위반 ...,https://news.joins.com/article/olink/23642697,NaN,경찰 로고. 뉴스1 전남 지역 한 경찰서장과 간부들이 평일에 골프를 ...,0,중립
4,1.100901e+06,20210430,중앙일보,정은혜(jeong.eunhye1@joongang.co.kr),"화이자, EMA에 ""백신 접종 대상 12 15세까지 확대"" 요청",미분류,NaN,NaN,NaN,NaN,...,"위구르,독일,미국","EU,유럽의약품청,바이오엔테크사,EMA,AFP통신,유럽연합,제약사,화이자","EMA,대상,백신,접종,15세,확대,요청,화이자,미국,제약사,독일,바이오엔테크사,유...","미국,바이오엔테크사,15세,제약사,ema,유럽,eu,유럽의약품청,의약품청,연령대,유...",미국 제약사 화이자와 독일 바이오엔테크사가 유럽의약품청(EMA)에 자사 개발 백신(...,https://news.joins.com/article/olink/23642696,NaN,미국 제약사 화이자와 독일 바이오엔테크사가 유럽의약품청(EMA)에 자사 개발 백신(...,0,중립
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7947,1.100901e+06,20210401,중앙일보,정진우(dino87@joongang.co.kr),대만 코앞서 열리는 한 중외교회담,정치>외교,정치>청와대,국제>중국,NaN,NaN,...,"미국,일본,중국,헤네시닐랜드,팔라우,한국,태평양,한반도,중,장교,인도,아나폴리스,미...","국가안보국,중국,해군,한국,국가안보실,한·중외교회,정부,해군사관학교,청와대,한·미 ...","대만,코앞,중외교회담,중국,미국,장소,대결,상징,미국선,안보실장,협의,해군,쿼드,핵...","중국,대만,미국,샤먼,백악관,청와대,쿼드,일본,정의용,외교부,한국,아나폴리스,해병대...",바이든 정부 출범 이후 미 중 갈등이 고조되는 상황에서 정부가 외교 고차방정식을 풀...,https://news.joins.com/article/olink/23619867,NaN,중국은 1958년 8~10월 푸젠(福建)성 샤먼(廈門)시에서 약 1.8㎞ 떨어진 진...,-1,-부정
7948,1.100901e+06,20210401,중앙일보,김홍범(kim.hongbum@joongang.co.kr),화이자 “12~15세 예방효과 100%” 청소년 접종 길 열린다,사회>의료_건강,IT_과학>IT_과학일반,NaN,NaN,NaN,...,"미국소아과학회,최고경영자,미국","바이오엔테크,FDA,화이자,식품의약국,AP 통신","100%,예방,효과,100%,청소년,접종,결과,미국,임상,결과,수주,허가,신청,가을...","미국,코로나19,월스트리트저널,불라,앨버트,경영자,의약국,6개월,11세,최고경영자,...",화이자 바이오엔테크의 코로나19 백신이 12 15세 청소년에게 100%의 예방 효과...,https://news.joins.com/article/olink/23619866,NaN,화이자·바이오엔테크의 코로나19 백신이 12∼15세 청소년에게 100%의 예방 효과...,0,중립
7949,1.100901e+06,20210401,중앙일보,강찬호(stoncold@joongang.co.kr),"“김정숙 여사가 끌어안길래, 밀어내며 쓴소리했다”",정치>외교,정치>청와대,NaN,NaN,NaN,...,"서해,김,천안,하길,북한,평택,평창","정찰총국,사령부,정부,중앙일보,북한,해군","김정숙,여사,쓴소리,아들,천안함,폭침,윤청자씨,북한,도발,숨기,묻자,미안,사령부,평...","추모식,천안함,옆자리,쓴소리,숨기,윤청자,평창,서해수호,평택,민평기,문재인,보훈처장...",지난 26일 평택 해군 2함대 사령부에서 열린 ‘제6회 서해수호의 날’ 기념식에서 ...,https://news.joins.com/article/olink/23619864,NaN,김정숙 여사가 26일 오후 경기도 평택시 해군 2함대사령부 에서 열린 제6회 서해수...,-1,-부정
7950,1.100901e+06,20210401,중앙일보,신성식.김민욱(ssshin@joongang.co.kr),"2030은 설암, 60대는 편도암 늘었다",사회>의료_건강,지역>경기,지역>충남,NaN,NaN,...,"설암,구강","보건복지부,중앙암등록본부,이비인후과,국립암센터,세계보건기구,WHO","설암,60대,편도암,국립암센터,중앙암,등록,통계,요인,유전,20,,설암,혀암,60대...","편도암,설암,연평균,국립암센터,60대,정유석,국가암검진사업,자궁경부암,헬리코박터균,...","20, 30대는 설암(혀암), 60대 이상은 편도암의 증가세가 두드러진 것으로 나타...",http

In [44]:
df = pd.read_excel('C:/Develops/newspapers/중앙일보/j_press_combine.xls', names=['identical', 'date', 'press', 'name', 'title', 'c1', 'c2', 'c3', 'a1', 'a2', 'a3', 'person', 'place', 'institute', 'keyword', 'topkeyword', 'body', 'url', 'tf', 'total_body', 'judge', 'judge_ko'])
df

,identical,date,press,name,title,c1,c2,c3,a1,a2,...,place,institute,keyword,topkeyword,body,url,tf,total_body,judge,judge_ko
0,1.100901e+06,20210430,중앙일보,김효경(kaypubb@joongang.co.kr),"하주석 5안타 6타점, 한화 롯데 꺾고 4연패 탈출",스포츠>야구,스포츠>농구_배구,스포츠>축구,NaN,NaN,...,"대구,경기,사직구장,부산","한화 이글스,삼성 라이온즈,한화,롯데,LG 트윈스,라이온,롯데 자이언츠,LG,삼성","6타점,하주석,한화,롯데,탈출,무실점,윤호솔,이닝,9년,프로,이글스,프로,야구,한화...","하주석,한화,롯데,삼성,무실점,적시타,윤호솔,5안타,10패,4연패,2회,노시환,6타...",프로야구 한화 이글스가 하주석의 활약에 힘입어 롯데 자이언츠에 재역전승을 거뒀다. ...,https://news.joins.com/article/olink/23642699,NaN,30일 롯데전에서 결승타를 때려내는 한화 하주석. [연합뉴스] 프로야...,-1,중립
1,1.100901e+06,20210430,중앙일보,한영혜(han.younghye@joongang.co.kr),"아파트 내서 '2m' 음주운전 1500만원 벌금폭탄 맞았다, 왜",사회>장애인,NaN,NaN,NaN,NaN,...,울산,"울산지법,형사5단독,재판부","2m,아파트,내서,음주운전,1500만,벌금폭탄,단지,아파트,가량,차량,운전,혐의,기...","음주운전,울산지법,a씨,교통사고,회사원,1500만,내서,김정철,혈중알콜농도,알코올농...",술을 마신 채 아파트 단지 내에서 2m가량 차량을 운전한 혐의로 기소된 50대 회사...,https://news.joins.com/article/olink/23642698,NaN,사진 크게보기 음주운전 단속 현장. 연합뉴스 술을 마신 채 아파트 단...,0,중립
2,1.100901e+06,20210430,중앙일보,김준희(kim.junhee@joongang.co.kr),회식 모임 금지했는데 나주경찰서장과 간부 3명 골프 쳤다,지역>강원,지역>대전,지역>전남,NaN,NaN,...,"경찰서장,전남,나주","경찰서,전남경찰청,정부,영암,감찰계,나주경찰서장","회식,모임,나주경찰서장,골프,간부,전남경찰청,확인,전남경찰청,감찰,착수,전남,지역,...","전남경찰청,전남,나주,코로나바이러스,경찰서,나주경찰서장,3명,간부들,공무원들,사실관...",전남 지역 한 경찰서장과 간부들이 평일에 골프를 친 것으로 드러나 복무 지침 위반 ...,https://news.joins.com/article/olink/23642697,NaN,경찰 로고. 뉴스1 전남 지역 한 경찰서장과 간부들이 평일에 골프를 ...,0,중립
3,1.100901e+06,20210430,중앙일보,정은혜(jeong.eunhye1@joongang.co.kr),"화이자, EMA에 ""백신 접종 대상 12 15세까지 확대"" 요청",미분류,NaN,NaN,NaN,NaN,...,"위구르,독일,미국","EU,유럽의약품청,바이오엔테크사,EMA,AFP통신,유럽연합,제약사,화이자","EMA,대상,백신,접종,15세,확대,요청,화이자,미국,제약사,독일,바이오엔테크사,유...","미국,바이오엔테크사,15세,제약사,ema,유럽,eu,유럽의약품청,의약품청,연령대,유...",미국 제약사 화이자와 독일 바이오엔테크사가 유럽의약품청(EMA)에 자사 개발 백신(...,https://news.joins.com/article/olink/23642696,NaN,미국 제약사 화이자와 독일 바이오엔테크사가 유럽의약품청(EMA)에 자사 개발 백신(...,0,중립
4,1.100901e+06,20210430,중앙일보,나운채(na.unchae@joongang.co.kr),여고 사물함에 토끼 사체 CCTV 찍힌 女용의자 긴급체포,사회>사건_사고,사회>교육_시험,지역>부산,NaN,NaN,...,"제주시,제주","동부경찰서,제주,경찰","사물,토끼,사체,CCTV,용의자,긴급체포,제주,사물함,부패,토끼,사체,용의자,경찰,...","제주,용의자,사물함,긴급체포,a씨,제주시,경찰서,건조물,건조물침입,폐쇄회로,비닐봉지...",제주의 한 여고 사물함에 부패한 토끼 사체를 두고 사라진 용의자가 경찰에 붙잡혔다....,https://news.joins.com/article/olink/23642695,NaN,제주 동부경찰서. 연합뉴스 제주의 한 여고 사물함에 부패한 토끼 사체...,-1,부정
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7946,1.100901e+06,20210401,중앙일보,정진우(dino87@joongang.co.kr),대만 코앞서 열리는 한 중외교회담,정치>외교,정치>청와대,국제>중국,NaN,NaN,...,"미국,일본,중국,헤네시닐랜드,팔라우,한국,태평양,한반도,중,장교,인도,아나폴리스,미...","국가안보국,중국,해군,한국,국가안보실,한·중외교회,정부,해군사관학교,청와대,한·미 ...","대만,코앞,중외교회담,중국,미국,장소,대결,상징,미국선,안보실장,협의,해군,쿼드,핵...","중국,대만,미국,샤먼,백악관,청와대,쿼드,일본,정의용,외교부,한국,아나폴리스,해병대...",바이든 정부 출범 이후 미 중 갈등이 고조되는 상황에서 정부가 외교 고차방정식을 풀...,https://news.joins.com/article/olink/23619867,NaN,중국은 1958년 8~10월 푸젠(福建)성 샤먼(廈門)시에서 약 1.8㎞ 떨어진 진...,-1,부정
7947,1.100901e+06,20210401,중앙일보,김홍범(kim.hongbum@joongang.co.kr),화이자 “12~15세 예방효과 100%” 청소년 접종 길 열린다,사회>의료_건강,IT_과학>IT_과학일반,NaN,NaN,NaN,...,"미국소아과학회,최고경영자,미국","바이오엔테크,FDA,화이자,식품의약국,AP 통신","100%,예방,효과,100%,청소년,접종,결과,미국,임상,결과,수주,허가,신청,가을...","미국,코로나19,월스트리트저널,불라,앨버트,경영자,의약국,6개월,11세,최고경영자,...",화이자 바이오엔테크의 코로나19 백신이 12 15세 청소년에게 100%의 예방 효과...,https://news.joins.com/article/olink/23619866,NaN,화이자·바이오엔테크의 코로나19 백신이 12∼15세 청소년에게 100%의 예방 효과...,0,중립
7948,1.100901e+06,20210401,중앙일보,강찬호(stoncold@joongang.co.kr),"“김정숙 여사가 끌어안길래, 밀어내며 쓴소리했다”",정치>외교,정치>청와대,NaN,NaN,NaN,...,"서해,김,천안,하길,북한,평택,평창","정찰총국,사령부,정부,중앙일보,북한,해군","김정숙,여사,쓴소리,아들,천안함,폭침,윤청자씨,북한,도발,숨기,묻자,미안,사령부,평...","추모식,천안함,옆자리,쓴소리,숨기,윤청자,평창,서해수호,평택,민평기,문재인,보훈처장...",지난 26일 평택 해군 2함대 사령부에서 열린 ‘제6회 서해수호의 날’ 기념식에서 ...,https://news.joins.com/article/olink/23619864,NaN,김정숙 여사가 26일 오후 경기도 평택시 해군 2함대사령부 에서 열린 제6회 서해수...,-1,부정
7949,1.100901e+06,20210401,중앙일보,신성식.김민욱(ssshin@joongang.co.kr),"2030은 설암, 60대는 편도암 늘었다",사회>의료_건강,지역>경기,지역>충남,NaN,NaN,...,"설암,구강","보건복지부,중앙암등록본부,이비인후과,국립암센터,세계보건기구,WHO","설암,60대,편도암,국립암센터,중앙암,등록,통계,요인,유전,20,,설암,혀암,60대...","편도암,설암,연평균,국립암센터,60대,정유석,국가암검진사업,자궁경부암,헬리코박터균,...","20, 30대는 설암(혀암), 60대 이상은 편도암의 증가세가 두드러진 것으로 나타...",https://news.joins.co

In [34]:
df.to_excel('C:/Develops/newspapers/중앙일보/j_press_combine.xls', header=None,index=False)

In [51]:
db_news = sqlite3.connect('C:/Develops/newspapers/중앙일보/reset.db')
c = db_news.cursor()

In [52]:
c.execute("CREATE TABLE newspapers (id INTEGER PRIMARY KEY AUTOINCREMENT, identical TEXT, date TEXT, press TEXT, name TEXT, title TEXT, c1 TEXT, c2 TEXT, c3 TEXT, a1 TEXT, a2 TEXT, a3 TEXT, person TEXT, place TEXT, institute TEXT, keyword TEXT, topkeyword TEXT, body TEXT, url TEXT, tf TEXT, total_body TEXT, judge TEXT, judge_ko TEXT)")

In [53]:
for row in df.iterrows():
    identical = row[1][0]
    date = row[1][1]
    press = row[1][2]
    name = row[1][3]
    title = row[1][4]
    c1 = row[1][5]
    c2 = row[1][6]
    c3 = row[1][7]
    a1 = row[1][8]
    a2 = row[1][9]
    a3 = row[1][10]
    person = row[1][11]
    place = row[1][12]
    institute = row[1][13]
    keyword = row[1][14]
    topkeyword = row[1][15]
    body = row[1][16]
    url = row[1][17]
    tf = row[1][18]
    total_body = row[1][19]
    judge = row[1][20]
    judge_ko = row[1][21]
    c.execute("INSERT INTO newspapers (identical, date, press, name, title, c1, c2, c3, a1, a2, a3, person, place, institute, keyword, topkeyword, body, url, tf, total_body, judge, judge_ko) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) ",(identical, date, press, name, title, c1, c2, c3, a1, a2, a3, person, place, institute, keyword, topkeyword, body, url, tf, total_body, judge, judge_ko))
db_news.commit()

In [50]:
db_news.close()

In [ ]:
pos_dict
neg_dict


In [ ]:
----------------------------------------------------------------

In [ ]:
, names=['identical', 'date', 'press', 'name', 'title', 'c1', 'c2', 'c3', 'a1', 'a2', 'a3', 'person', 'place', 'institute', 'keyword', 'topkeyword', 'body', 'url', 'tf', 'total_body', 'judge', 'judge_ko', 'pos_dict', 'neg_dict']